# AOS Observation Run - March 10th 2023

This notebook contains the execution of the SITCOM-615 test described in https://jira.lsstcorp.org/browse/SITCOM-615, expected to run during the 2023-03B AuxTel Observing Run. 
                
This test assumes you're on-target and already at the focus position. <br>
It should be run directly after SITCOM-606

-----
## Setup

### Import libraries

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_wep_align import LatissWEPAlign, LatissIntraExtraFocalData
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

### Setting up logger

In [ ]:
logger = logging.getLogger("SITCOM-615")
logger.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [ ]:
script = LatissIntraExtraFocalData(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

In [ ]:
latiss.read_out_time

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [1]:
script.log.info(f'START- SUMMIT-7280 Z-axis sensitivity Matrix data -- at {Time.now()}')

NameError: name 'script' is not defined

In [ ]:
def run_script(configuration):
    await script.set_state(ScriptState.UNCONFIGURED)

    config_data = script.cmd_configure.DataType()
    config_data.config = configuration
    await script.do_configure(config_data)

    script.intra_visit_id = None
    script.extra_visit_id = None
    script.short_timeout = 10

    group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

    await script.do_setGroupId(group_id_data)

    await script.arun()

----
## Assess that conditions meet criteria
Seeing must be stable and less than ~1.2 arcsec.

### Sensitivity Matrix

#### Offsets x

In [ ]:
offsets_x = np.linspace(-2.0,2.0, 10) # mm

for offset_x in offsets_x:
    configuration = yaml.safe_dump({"filter": "SDSSr_65mm", 
                                    "grating": 'empty_1',
                                    "exposure_time": 20,
                                    "offset_x": offset_x})
    
    await run_script(configuration)


#### Offsets Y

In [ ]:
offsets_y = np.linspace(-2.0,2.0, 10) # mm

for offset_y in offsets_y:
    configuration = yaml.safe_dump({"filter": "SDSSr_65mm", 
                                    "grating": 'empty_1',
                                    "exposure_time": 20,
                                    "program" : "SITCOM-615", 
                                    "offset_y": offset_y})
    await run_script(configuration)
    

In [ ]:
script.log.info(f'END- SITCOM-615 Z-axis sensitivity Matrix data -- at {Time.now()}')